In [8]:
import os
# Add directory for NVIDIA gpu. Ignore if not Windows
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")

<AddedDllDirectory('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin')>

In [9]:
import tensorflow as tf
# print(tf.reduce_sum(tf.random.normal([1000, 1000])))
# Confirm tensorflow running on gpu
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Resizing
import numpy as np
import pickle as pkl

### Base CovNet Model

In [11]:
# function to build the conv net base


# complete this function
def build_base_convnet_model():
    """Re-create the model from the first prompt, but with a different input shape"""
    
    # Return this variable
    model = None
    
    # YOUR CODE HERE
    inputs = keras.Input(shape = (299, 299, 3))
    x = keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')(inputs)
    x = keras.layers.MaxPooling2D(pool_size = 2)(x)
    x = keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')(x)
    x = keras.layers.MaxPooling2D(pool_size = 2)(x)
    x = keras.layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu')(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(inputs, outputs)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

def load_image_data(base_path: str) -> tuple:
    """Write a function that accepts a base path that contains all of the directories, and creates training,
    validation and test sets"""
    
    # Return these variables from the function
    train_data = keras.utils.image_dataset_from_directory(f'{base_path}/train', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)

    validation_data = keras.utils.image_dataset_from_directory(f'{base_path}/val', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)
    
    test_data = keras.utils.image_dataset_from_directory(f'{base_path}/test', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)
    
    # YOUR CODE HERE
    
    
    return train_data, validation_data, test_data

def fit_convnet_model(model, train_set, validation_set):
    """Fit a model with the above stated criteria"""
    early_stopping = keras.callbacks.EarlyStopping(patience = 10)
    
    # YOUR CODE HERE
    model.fit(train_set, 
              validation_data = validation_set, 
              callbacks = [early_stopping], 
              epochs = 500)
    
    return model

In [12]:
# run this cell to create your base convolution model and training, validation & test sets
base_mod = build_base_convnet_model()
train_data, validation_data, test_data = load_image_data('../data/COVID-19_Radiography_Dataset/TwoClasses/split')

Found 8284 files belonging to 2 classes.
Found 2761 files belonging to 2 classes.
Found 2763 files belonging to 2 classes.


In [13]:
fitted_model = fit_convnet_model(base_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 29s 85ms/step - loss: 64.6308 - accuracy: 0.7639 - val_loss: 0.8445 - val_accuracy: 0.8421
Epoch 2/500
259/259 [==============================] - 21s 79ms/step - loss: 11.1127 - accuracy: 0.8154 - val_loss: 0.7161 - val_accuracy: 0.8526
Epoch 3/500
259/259 [==============================] - 21s 80ms/step - loss: 12.4725 - accuracy: 0.8334 - val_loss: 1.0165 - val_accuracy: 0.8606
Epoch 4/500
259/259 [==============================] - 21s 80ms/step - loss: 15.9807 - accuracy: 0.8712 - val_loss: 0.5563 - val_accuracy: 0.8888
Epoch 5/500
259/259 [==============================] - 21s 82ms/step - loss: 2.1626 - accuracy: 0.8891 - val_loss: 0.6703 - val_accuracy: 0.8968
Epoch 6/500
259/259 [==============================] - 20s 78ms/step - loss: 0.7264 - accuracy: 0.9166 - val_loss: 0.6433 - val_accuracy: 0.9044
Epoch 7/500
259/259 [==============================] - 20s 78ms/step - loss: 0.5368 - accuracy: 0.9357 - val_loss: 0.7448 - va

### Inception V3 Model



In [14]:
from keras.applications import InceptionV3

In [15]:
def build_transfer_learning_model():
    
    # return this variable
    model = None

    model_input = keras.Input(shape = (299, 299, 3))
    
    base_model = InceptionV3(input_shape = (299, 299, 3), weights='imagenet', include_top=False)

    # make the weights in the base model non-trainable
    for layer in base_model.layers:
      layer.trainable = False

    # combine the base model with a dense layer and output layer for the 10 classes
    # the preprocess_input transforms input data according to how the model was trained
    
    x = keras.applications.inception_v3.preprocess_input(model_input)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(model_input, output)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [16]:
transfer_learning_mod = build_transfer_learning_model()

In [17]:
inception_model = fit_convnet_model(transfer_learning_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 34s 116ms/step - loss: 4.7162 - accuracy: 0.8373 - val_loss: 0.2678 - val_accuracy: 0.9152
Epoch 2/500
259/259 [==============================] - 29s 110ms/step - loss: 0.3598 - accuracy: 0.9108 - val_loss: 0.1571 - val_accuracy: 0.9518
Epoch 3/500
259/259 [==============================] - 29s 111ms/step - loss: 0.2969 - accuracy: 0.9284 - val_loss: 0.8481 - val_accuracy: 0.8758
Epoch 4/500
259/259 [==============================] - 29s 111ms/step - loss: 0.2546 - accuracy: 0.9454 - val_loss: 0.2162 - val_accuracy: 0.9384
Epoch 5/500
259/259 [==============================] - 29s 111ms/step - loss: 0.1661 - accuracy: 0.9507 - val_loss: 0.1585 - val_accuracy: 0.9627
Epoch 6/500
259/259 [==============================] - 29s 110ms/step - loss: 0.1468 - accuracy: 0.9604 - val_loss: 0.1519 - val_accuracy: 0.9612
Epoch 7/500
259/259 [==============================] - 29s 113ms/step - loss: 0.1284 - accuracy: 0.9623 - val_loss: 0.1456 -

### ResNet Model

In [18]:
from keras.applications import ResNet50

In [19]:
def build_transfer_learning_model():
    
    # return this variable
    model = None

    model_input = keras.Input(shape = (299, 299, 3))
    
    base_model = ResNet50(input_shape = (299, 299, 3), weights='imagenet', include_top=False)

    # make the weights in the base model non-trainable
    for layer in base_model.layers:
      layer.trainable = False

    # combine the base model with a dense layer and output layer for the 10 classes
    # the preprocess_input transforms input data according to how the model was trained
    
    x = keras.applications.resnet50.preprocess_input(model_input)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(model_input, output)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [20]:
transfer_learning_mod = build_transfer_learning_model()

In [21]:
resnet_model = fit_convnet_model(transfer_learning_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 34s 119ms/step - loss: 6.7822 - accuracy: 0.8854 - val_loss: 0.3513 - val_accuracy: 0.9464
Epoch 2/500
259/259 [==============================] - 29s 112ms/step - loss: 0.4130 - accuracy: 0.9372 - val_loss: 0.1942 - val_accuracy: 0.9634
Epoch 3/500
259/259 [==============================] - 30s 114ms/step - loss: 0.3029 - accuracy: 0.9567 - val_loss: 0.1893 - val_accuracy: 0.9667
Epoch 4/500
259/259 [==============================] - 29s 113ms/step - loss: 0.1809 - accuracy: 0.9666 - val_loss: 0.2687 - val_accuracy: 0.9663
Epoch 5/500
259/259 [==============================] - 29s 113ms/step - loss: 0.1803 - accuracy: 0.9724 - val_loss: 0.1979 - val_accuracy: 0.9739
Epoch 6/500
259/259 [==============================] - 29s 113ms/step - loss: 0.1359 - accuracy: 0.9754 - val_loss: 0.2158 - val_accuracy: 0.9779
Epoch 7/500
259/259 [==============================] - 29s 113ms/step - loss: 0.1353 - accuracy: 0.9788 - val_loss: 0.2454 -

In [22]:
results = resnet_model.evaluate(test_data)

87/87 [==============================] - 7s 80ms/step - loss: 0.8157 - accuracy: 0.9504


In [23]:
predictions = resnet_model.predict(test_data)
predictions

87/87 [==============================] - 7s 76ms/step


array([[1.000000e+00],
       [1.000000e+00],
       [0.000000e+00],
       ...,
       [1.000000e+00],
       [5.972717e-28],
       [9.714957e-01]], dtype=float32)

### ResNet with Dropout 0.25

In [24]:
def build_transfer_learning_model():
    
    # return this variable
    model = None

    model_input = keras.Input(shape = (299, 299, 3))
    
    base_model = ResNet50(input_shape = (299, 299, 3), weights='imagenet', include_top=False)

    # make the weights in the base model non-trainable
    for layer in base_model.layers:
      layer.trainable = False

    # combine the base model with a dense layer and output layer for the 10 classes
    # the preprocess_input transforms input data according to how the model was trained
    
    x = keras.applications.resnet50.preprocess_input(model_input)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.25)(x)
    output = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(model_input, output)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [25]:
transfer_learning_mod = build_transfer_learning_model()

In [26]:
resnet25_model = fit_convnet_model(transfer_learning_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 33s 116ms/step - loss: 4.4181 - accuracy: 0.8876 - val_loss: 0.1618 - val_accuracy: 0.9576
Epoch 2/500
259/259 [==============================] - 29s 112ms/step - loss: 0.2038 - accuracy: 0.9450 - val_loss: 0.1403 - val_accuracy: 0.9649
Epoch 3/500
259/259 [==============================] - 29s 111ms/step - loss: 0.1681 - accuracy: 0.9616 - val_loss: 0.1105 - val_accuracy: 0.9754
Epoch 4/500
259/259 [==============================] - 29s 112ms/step - loss: 0.1318 - accuracy: 0.9683 - val_loss: 0.1379 - val_accuracy: 0.9703
Epoch 5/500
259/259 [==============================] - 29s 112ms/step - loss: 0.0878 - accuracy: 0.9763 - val_loss: 0.1571 - val_accuracy: 0.9750
Epoch 6/500
259/259 [==============================] - 29s 111ms/step - loss: 0.0923 - accuracy: 0.9790 - val_loss: 0.1298 - val_accuracy: 0.9754
Epoch 7/500
259/259 [==============================] - 29s 111ms/step - loss: 0.0710 - accuracy: 0.9824 - val_loss: 0.1430 -

In [27]:
results = resnet25_model.evaluate(test_data)

87/87 [==============================] - 7s 78ms/step - loss: 0.1093 - accuracy: 0.9844


In [28]:
predictions = resnet25_model.predict(test_data)
predictions

87/87 [==============================] - 7s 75ms/step


array([[1.000000e+00],
       [1.000000e+00],
       [1.000000e+00],
       ...,
       [1.000000e+00],
       [5.434097e-27],
       [6.299504e-27]], dtype=float32)

### Final Model

In [ ]:
print("predictions shape:", predictions.shape)

In [ ]:
dict(zip(resnet_model.metrics_names, predictions))

In [ ]:
# Concatenate train and validation data
fulltrain = train_data.concatenate(validation_data)

In [ ]:
# Fit the model on 80% data
final_model = resnet_model.fit(fulltrain)

In [ ]:
filename = 'final_model.pkl'
pkl.dump(final_model, open(filename, 'wb'))

In [ ]:
import joblib
filename = 'final_model.pkl'
joblib.dump(final_model, filename)